DATE: 26th Oct 2020

This file will iterate over all the data files. Take one chunk of allData and then iterate over the rows to dientify province matches ...It will also read the HSN HISCO codes and update it in the respective columns


1. Read allData_chunk{idx}.csv
2. Iterate through rows and check for allLocation_chunks
3. Update the location information 
4. Iterate through the tweet text & update the HISCO codes
5. Save the file to Gdrive

HISCO codes (for single file) was done in the following colab notebook
Previous version can be found here:

https://colab.research.google.com/drive/1Ja5HNwJxR9pyDLcqizpi475AT9O41glY#scrollTo=LwCAd5y93e6L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/"My Drive"/"Dutch Tweets" .

cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk6.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk9.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk0.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk1.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk2.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk3.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk4.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk5.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk8.gsheet' for reading: Operation not supported
cp: cannot open 'dr

In [ ]:
# Read and concatenate all the location data 
import pandas as pd 

allLocation = pd.DataFrame()

for idx in range(10):
  try:
    allLocation = pd.concat([allLocation, pd.read_csv("Dutch Tweets/Location CSV/allLocation_chunk"+ str(idx) +".csv")], axis=0)
  except:
    print("Error encountered on the following chunk: ", idx)
    pass

print("Shape of allLocation dataframe: ", allLocation.shape)
allLocation.columns

Shape of allLocation dataframe:  (14809, 8)


Index(['Unnamed: 0', 'location', 'point_info', 'point', 'latitude',
       'longitude', 'altitude', 'Province'],
      dtype='object')

In [ ]:
# Preprocess the location file by updating the string information

allLocation.drop(columns=['Unnamed: 0'], inplace=True)
allLocation.reset_index(inplace=True, drop=True)
allLocation['location'] = allLocation['location'].astype("str")
allLocation.head(15)

,location,point_info,point,latitude,longitude,altitude,Province
0,Netherlands,Nederland,"(52.5001698, 5.7480821, 0.0)",52.500170,5.748082,0.0,Flevoland
1,"Amsterdam, The Netherlands","Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.372760,4.893604,0.0,Noord-Holland
2,nan,"Nanno, Comunità della Val di Non, Provincia di...","(46.3144754, 11.0480288, 0.0)",46.314475,11.048029,0.0,False
3,Amsterdam,"Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.372760,4.893604,0.0,Noord-Holland
4,The Netherlands,Nederland,"(52.5001698, 5.7480821, 0.0)",52.500170,5.748082,0.0,Flevoland
5,NLD-DEU,"Aeropuerto Intl. Quetzalcóatl, Anzures, Fracci...","(27.442489899999998, -99.56709906841328, 0.0)",27.442490,-99.567099,0.0,False
6,Lopik / Holland,"Lopik, Utrecht, Nederland","(51.9752848, 4.9491743, 0.0)",51.975285,4.949174,0.0,Utrecht
7,"Groningen, Nederland","Groningen, Nederland","(53.2190652, 6.5680077, 0.0)",53.219065,6.568008,0.0,Groningen
8,"Utrecht, Netherlands","Utrecht, Nederland","(52.0949753, 5.109708, 0.0)",52.094975,5.109708,0.0,Utrecht
9,Utrecht,"Utrecht, Nederland","(52.0949753, 5.109708, 0.0)",52.094975,5.109708,0.0,Utrecht


In [ ]:
## Read the HSN HISCO codes and process it 

# Read the HSNHISCO code file
hsnhisco = pd.read_excel("Dutch Tweets/hsnhisco.xlsx")
hsnhisco.drop_duplicates(subset=['Standard', 'HISCO'], inplace=True)
print(hsnhisco.shape)
hsnhisco.head()

# Create a Series of unique HSN HISCO codes
# And then extract list of all codes 

hsnhisco_dict = pd.Series(hsnhisco['HISCO'].values,index=hsnhisco['Standard']).to_dict()
print(hsnhisco.shape)
hisco_list = list(set(list(hsnhisco['Standard'])))
print(len(hisco_list))

(34257, 15)
(34257, 15)
34257


Read the data files

In [ ]:
# ## For debugging next code blocks

idx =5
allData_chunk = pd.read_excel("Dutch Tweets/Tweets Excel Version_v1/allDataList_chunk"+str(idx) + ".xlsx", dtype={'full_text': 'str', 'text Translation': 'str', 
                                                                                                        'description':'str','desc Translation': 'str', 
                                                                                                        'location': 'str'})
allData_chunk.reset_index(inplace=True, drop=True)
allData_chunk['location'] = allData_chunk['location'].astype("str")
allData_chunk.head()

In [ ]:
# allData = pd.merge(allData_chunk, allLocation,how='left', on= ['location'], suffixes=("_data","_loc") )
# allData.head()

In [ ]:
# print(f"Saving chunk to file: allData_chunk{idx}.xlsx" )

Saving chunk to file: allData_chunk0.xlsx


In [ ]:
# read each allData chunk & merge with allLocation file. 
# write it back to local disk

for idx in range(3, 10):
  print(f"Processing chunk: {idx}")
  try:
    allData_chunk = pd.read_excel("Dutch Tweets/Tweets Excel Version_v1/allDataList_chunk"+str(idx) + ".xlsx", dtype={'full_text': 'str', 'text Translation': 'str', 
                                                                                                        'description':'str','desc Translation': 'str', 
                                                                                                        'location': 'str'})
  except:
    print("Error process chunk: ", idx)
    pass
  allData_chunk.reset_index(inplace=True, drop=True)
  allData_chunk['location'] = allData_chunk['location'].astype("str")
  
  allData = pd.merge(allData_chunk, allLocation,how='left', on= ['location'], suffixes=("_data","_loc") )

  # apply the hisco code & standard
  allData['hisco_standard'] = allData['full_text'].apply(lambda tweet: [str(i) for i in hisco_list if i in str(tweet)])
  allData['hisco_code'] = allData['full_text'].apply(lambda tweet: [str(hsnhisco_dict[i]) for i in hisco_list if i in str(tweet)])

  # process the results, to ensure that the list brackets are removed    
  allData['hisco_standard'] = allData['hisco_standard'].apply(lambda text: str(text)[2:len(str(text))-2] ) 
  allData['hisco_code'] = allData['hisco_code'].apply(lambda text: str(text)[2:len(str(text))-2] )

  print(f"Saving chunk to file: allData_chunk{idx}.xlsx" )
  allData.to_excel("allData_chunk"+ str(idx) + ".xlsx", index=False)

Processing chunk: 3
Saving chunk to file: allData_chunk3.xlsx
Processing chunk: 4
Saving chunk to file: allData_chunk4.xlsx
Processing chunk: 5
Saving chunk to file: allData_chunk5.xlsx
Processing chunk: 6
Saving chunk to file: allData_chunk6.xlsx
Processing chunk: 7
Saving chunk to file: allData_chunk7.xlsx
Processing chunk: 8
Saving chunk to file: allData_chunk8.xlsx
Processing chunk: 9
Saving chunk to file: allData_chunk9.xlsx


In [ ]:
!cp *.xlsx drive/"My Drive"/"Dutch Tweets"/"Tweets Excel_v3"

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [ ]:
for idx in range(10):
  try:
    allData =pd.read_excel("allData_chunk"+str(idx)+".xlsx")
    print(f'Shape of data chunk {idx} :: {allData.shape}')
  except Exception as err:
    print(f"Error processing chun {idx}")
    print(err)
    pass
  
  

Shape of data chunk 0 :: (27019, 19)
Shape of data chunk 1 :: (27142, 19)
Shape of data chunk 2 :: (27130, 19)
Shape of data chunk 3 :: (27112, 19)
Shape of data chunk 4 :: (27217, 19)
Shape of data chunk 5 :: (27104, 19)
Shape of data chunk 6 :: (27026, 19)
Shape of data chunk 7 :: (27234, 19)
Shape of data chunk 8 :: (27137, 19)
Shape of data chunk 9 :: (27221, 18)


In [ ]:
## For debugging next code block
idx = 0

allData =pd.read_excel("allData_chunk"+str(idx)+".xlsx")
print(f'Shape of data chunk {idx} :: {allData.shape}')

Shape of data chunk 0 :: (27019, 19)


In [ ]:
allData.columns

Index(['full_text', 'text Translation', 'created_at', 'screen_name',
       'description', 'desc Translation', 'location', 'Weekday', 'Month',
       'Year', 'Day', 'point_info', 'point', 'latitude', 'longitude',
       'altitude', 'Province', 'hisco_standard', 'hisco_code'],
      dtype='object')

In [ ]:
# allData[allData['location'].isnull()]

In [ ]:
allData.loc[allData['location'].isnull(), 'point_info'] = ''
allData.loc[allData['location'].isnull(), 'point'] = ''
allData.loc[allData['location'].isnull(), 'latitude'] = ''
allData.loc[allData['location'].isnull(), 'longitude'] = ''

In [ ]:
allData.to_csv("allData_chunk"+ str(idx) + ".xlsx", index=False)

In [ ]:
for idx in range(10):
  try:
    allData =pd.read_excel("allData_chunk"+str(idx)+".xlsx")
    print(f'Shape of data chunk {idx} :: {allData.shape}.....Processing....')
    allData.loc[allData['location'].isnull(), 'point_info'] = ''
    allData.loc[allData['location'].isnull(), 'point'] = ''
    allData.loc[allData['location'].isnull(), 'latitude'] = ''
    allData.loc[allData['location'].isnull(), 'longitude'] = ''
    print(f'Saving file allData_chunk{idx}.xlsx')
    allData.to_csv("allData_chunk"+ str(idx) + ".xlsx", index=False)
  except Exception as err:
    print(f"Error processing chunk {idx}")
    print(err)
    pass
  
  

Error processing chun 0
Unsupported format, or corrupt file: Expected BOF record; found b'full_tex'
Shape of data chunk 1 :: (27142, 19).....Processing....
Saving file allData_chunk1.xlsx
Shape of data chunk 2 :: (27130, 19).....Processing....
Saving file allData_chunk2.xlsx
Shape of data chunk 3 :: (27112, 19).....Processing....
Saving file allData_chunk3.xlsx
Shape of data chunk 4 :: (27217, 19).....Processing....
Saving file allData_chunk4.xlsx
Shape of data chunk 5 :: (27104, 19).....Processing....
Saving file allData_chunk5.xlsx
Shape of data chunk 6 :: (27026, 19).....Processing....
Saving file allData_chunk6.xlsx
Shape of data chunk 7 :: (27234, 19).....Processing....
Saving file allData_chunk7.xlsx
Shape of data chunk 8 :: (27137, 19).....Processing....
Saving file allData_chunk8.xlsx
Shape of data chunk 9 :: (27221, 18).....Processing....
Saving file allData_chunk9.xlsx


In [ ]:
for idx in range(10):
  try:
    allData =pd.read_csv("allData_chunk"+str(idx)+".xlsx")
    print(f'Shape of data chunk {idx} :: {allData.shape}.....Processing....')
    allData.loc[~allData['hisco_code'].isnull(), 'Industry'] = True
    allData.loc[allData['hisco_code'].isnull(), 'Industry'] = False
    print(f'Saving file allData_chunk{idx}.xlsx')
    allData.to_csv("allData_chunk"+ str(idx) + ".xlsx", index=False)
  except Exception as err:
    print(f"Error processing chunk {idx}")
    print(err)
    pass
  
  

Shape of data chunk 0 :: (27019, 20).....Processing....
Saving file allData_chunk0.xlsx
Shape of data chunk 1 :: (27142, 20).....Processing....
Saving file allData_chunk1.xlsx
Shape of data chunk 2 :: (27130, 20).....Processing....
Saving file allData_chunk2.xlsx
Shape of data chunk 3 :: (27112, 20).....Processing....
Saving file allData_chunk3.xlsx
Shape of data chunk 4 :: (27217, 20).....Processing....
Saving file allData_chunk4.xlsx
Shape of data chunk 5 :: (27104, 20).....Processing....
Saving file allData_chunk5.xlsx
Shape of data chunk 6 :: (27026, 20).....Processing....
Saving file allData_chunk6.xlsx
Shape of data chunk 7 :: (27234, 20).....Processing....
Saving file allData_chunk7.xlsx
Shape of data chunk 8 :: (27137, 20).....Processing....
Saving file allData_chunk8.xlsx
Shape of data chunk 9 :: (27221, 19).....Processing....
Saving file allData_chunk9.xlsx


In [ ]:
allData.head(15)

,full_text,text Translation,created_at,screen_name,description,desc Translation,location,Weekday,Month,Year,point_info,point,latitude,longitude,altitude,Province,hisco_standard,hisco_code,Industry
0,1300 nieuwe besmettingen met corona: Het RIVM ...,1300 new infections with corona: RIVM reported...,Mon Sep 14 14:10:02 +0000 2020,assercourant,De nieuwssite van Assen en omstreken. https://...,The news of Assen and environs. https://t.co/4...,Assen,0,9,2020,"Assen, Drenthe, Nederland","(52.99037165, 6.553377770642328, 0.0)",52.990372,6.553378,0.0,Drenthe,NaN,NaN,False
1,1300 nieuwe besmettingen met corona: Het RIVM ...,1300 new infections with corona: RIVM reported...,Mon Sep 14 14:10:02 +0000 2020,assercourant,De nieuwssite van Assen en omstreken. https://...,The news of Assen and environs. https://t.co/4...,Assen,0,9,2020,"Assen, Drenthe, Nederland","(52.99037165, 6.553377770642328, 0.0)",52.990372,6.553378,0.0,Drenthe,NaN,NaN,False
2,1300 nieuwe besmettingen met corona: Het RIVM ...,1300 new infections with corona: RIVM reported...,Mon Sep 14 14:10:03 +0000 2020,CoevordenHAH,Nieuws uit de gemeente Coevorden. Officiële ac...,News from the municipality of Coevorden. Offic...,Coevorden,0,9,2020,"Coevorden, Drenthe, Nederland","(52.65266925, 6.739318887009727, 0.0)",52.652669,6.739319,0.0,Drenthe,NaN,NaN,False
3,RT @Variety: #Tenet likely to be delayed again...,RT @Variety: #Tenet likely to be delayed again...,Mon Jul 13 15:26:13 +0000 2020,BelleOog,Interviews in English with mainly Dutch film t...,Interviews in English with Mainly Dutch film t...,Amsterdam,0,7,2020,"Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.372760,4.893604,0.0,Noord-Holland,NaN,NaN,False
4,RT @Variety: #Tenet likely to be delayed again...,RT @Variety: #Tenet likely to be delayed again...,Mon Jul 13 15:26:13 +0000 2020,BelleOog,Interviews in English with mainly Dutch film t...,Interviews in English with Mainly Dutch film t...,Amsterdam,0,7,2020,"Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.372760,4.893604,0.0,Noord-Holland,NaN,NaN,False
5,1300 nieuwe besmettingen met corona: Het RIVM ...,1300 new infections with corona: RIVM reported...,Mon Sep 14 14:10:03 +0000 2020,Emmen_nu,Nieuws uit de gemeente Emmen. Emmer Courant is...,News from the municipality of Emmen. Bucket Co...,Emmen,0,9,2020,"Emmen, Drenthe, Nederland","(52.788937, 6.8939001, 0.0)",52.788937,6.893900,0.0,Drenthe,NaN,NaN,False
6,@3Groter @lotje_hx Ik denk dan óók dat die geh...,@ 3Groter @lotje_hx I ​​am thinking also that ...,Mon Sep 14 14:10:05 +0000 2020,Evelien_Netjes,De aarde staat niet stil. Niet #links niet #re...,The earth does not stand still. Not #links not...,Op de Zeebodem,0,9,2020,"Welkom op de zeebodem, Schokkerringweg, Ens, N...","(52.6398099, 5.8166619, 0.0)",52.639810,5.816662,0.0,Flevoland,nertsenfokker,61240,True
7,Anti lockdown protest Milaan:\n\nDe betogers e...,Anti lockdown protest Milan:\n\nThe protesters...,Wed Jun 03 15:42:24 +0000 2020,InwonerL,"!!AWAKENED!!\n\nPRO FVD, PVV AND FREEDOM OF SP...","Awakened !! !!\n\nPRO FVD, PVV AND FREEDOM OF ...",NaN,2,6,2020,NaN,NaN,NaN,NaN,0.0,False,NaN,NaN,False
8,Anti lockdown protest Milaan:\n\nDe betogers e...,Anti lockdown protest Milan:\n\nThe protesters...,Wed Jun 03 15:42:24 +0000 2020,InwonerL,"!!AWAKENED!!\n\nPRO FVD, PVV AND FREEDOM OF SP...","Awakened !! !!\n\nPRO FVD, PVV AND FREEDOM OF ...",NaN,2,6,2020,NaN,NaN,NaN,NaN,0.0,False,NaN,NaN,False
9,RT @nlvaarder: Rutte wil Gommers spreken over ...,RT @nlvaarder: Gommers Rutte wants to talk abo...,Mon Jul 13 15:26:42 +0000 2020,Klandestien010,Al vallen er 1000 aan mijn zijde en 10000 aan ...,May fall in 1000 by my side and 10000 on my ri...,Ergens in rotterdam,0,7,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
